# Pytorch 3x4x2 model 

Purpose of this notebook is to show Pytorch notation for building neural networks, as opposed to doing it using Keras/Tensorflow.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

n_input = 3 # number of features
n_hidden = 4 # number of hidden layers
n_out = 2 # number of outputs
batch_size = 64 # batch size for mini-batch gradient descent
learning_rate = 0.01

# load dataset
df = pd.read_csv('iris.csv')
X = df.iloc[:,:3].to_numpy() # only use 1st 3 features (not logical, but to mimic the 3x4x2 model in the slides)
y = df.iloc[:,-3:-1].to_numpy() # the 3rd and 2nd columns from the right are the label (not logical, but to mimic the 3x4x2 model in the slides)

# transform to pytorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# create model
model = nn.Sequential(nn.Linear(n_input, n_hidden),
                      nn.Tanh(), # activation function
                      nn.Linear(n_hidden, n_out),
                      nn.Sigmoid()) # activation function
loss_function = nn.BCELoss() # binary cross-entropy
optimizer = optim.SGD(model.parameters(), lr=learning_rate) # stochastic gradient descent

# visualize model architecture
print(model)

# train model
for epoch in range(5000): # go 5000 times through the complete dataset
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]

        y_pred = model(X_batch) # perform forward pass (== do a prediction)
        loss = loss_function(y_pred, y_batch) # calculate loss
        optimizer.zero_grad() # reset gradient to zero, otherwise the gradient of previous batches is added
        loss.backward() # compute the gradients of all weights and biases
        optimizer.step() # update all weights and biases
    print(f'\rfinished training of epoch {epoch}', end="")

# compute accuracy on *train* set (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)
accuracy1 = (y_pred[:,0].round() == y[:,0]).float().mean()
accuracy2 = (y_pred[:,1].round() == y[:,1]).float().mean()
print(f"\nAccuracy on each column: [{accuracy1}, {accuracy2}]")